In [178]:
# Leitura dos Microdadosimport pandas as pd
import numpy as np
import re
import pymupdf

from PyPDF2 import PdfReader

# Leitura dos Microdados

In [179]:
# função que realiza a leitura dos microdados

def reading_data(path_to_data: str, year: str) -> pd.DataFrame:

    df = pd.read_csv(path_to_data, sep=';', encoding='latin-1')
    
    df = df[(df['SG_AREA'] == 'CH') & (df['TX_COR'].str.lower() == 'azul')]
    df = df[df['CO_PROVA'] == df['CO_PROVA'].min()]
    df = df[['CO_POSICAO', 'TX_GABARITO', 'NU_PARAM_A', 'NU_PARAM_B', 'NU_PARAM_C']].copy()
    
    df['ANO'] = year

    return df

In [180]:
# Função que recebe o caminho do pdf, a página de início e a de fim e retorna as questões


def extract_questions(full_text: str, caps_lock: bool = False) -> list[str]:
    delimiter = "QUESTÃO " if caps_lock else "Questão "
    sections = full_text.split(delimiter)
    questions = [s[3:] for s in sections if s[:2].isnumeric()]
    return questions


def get_questions(pdf_path: str, start_page: int, end_page: int, caps_lock: bool = False) -> list[str]:
    reader = PdfReader(pdf_path)
    final_questions = []

    for page_num in range(start_page, end_page):
        full_text = reader.pages[page_num].extract_text() or ""
        final_questions.extend(extract_questions(full_text, caps_lock))

    return final_questions

In [181]:
# Função que retorna as questões sem as alternativas e as alternativas formatadas

def format_alt(alt: str) -> str:
    match = re.search(r"(A\s.+?)(B\s.+?)(C\s.+?)(D\s.+?)(E\s.+)", alt)
    if match:
        groups = match.groups()
        return '; '.join([f"{item[0]}: {item[2:-1]}" for item in groups])
    return ''

def get_alternatives(questions: list[str]) -> tuple[list[str], list[str]]:
    formatted_questions = []
    formatted_alternatives = []

    for question in questions:
        parts = re.split(r"(\nA\s.*\n)", question)

        alternatives_text = ''.join(parts[-2:]).replace('\n', '')
        alternatives_text = re.sub(r"\*.*\*", "", alternatives_text)
        alternatives_text = re.sub(r"(CH).*\d", "", alternatives_text)
        alternatives = format_alt(alternatives_text)

        question_text = ''.join(parts[:-2]).replace('\n', '')

        formatted_questions.append(question_text)
        formatted_alternatives.append(alternatives)

    return formatted_questions, formatted_alternatives

In [182]:
# Função auxiliar para criar o dataset com as novas colunas

def merge_quests_alts(dataset: pd.DataFrame, questions: list[str], alts: list[str]) -> pd.DataFrame:
    dataset['QUESTOES'] = questions
    dataset['ALTERNATIVAS'] = alts

    return dataset

# Leitura dos Microdados

In [183]:
df_2015 = reading_data('../data/microdados/microdados_2015.csv', '2015')

In [184]:
df_2016 = reading_data('../data/microdados/microdados_2015.csv', '2016')

In [185]:
df_2017 = reading_data('../data/microdados/microdados_2015.csv', '2017')

# Leituras da Provas

In [186]:
# Caminho da prova ENEM 2015
prova_2015 = '../data/provas/ENEM_2015.pdf'

# Caminho da prova ENEM 2016
prova_2016 = '../data/provas/ENEM_2016.pdf'

# Caminho da prova ENEM 2027
prova_2017 = '../data/provas/ENEM_2017.pdf'

In [187]:
questoes_2015 = get_questions(prova_2015, 1, 15, True)
len(questoes_2015)

45

In [188]:
questoes_2016 = get_questions(prova_2016, 1, 15, True)
len(questoes_2016)

45

In [189]:
questoes_2017 = get_questions(prova_2017, 19, 32, True)
len(questoes_2017)

45

## Alternativas


**2015**

In [190]:
questoes_2015, alts_2015 = get_alternatives(questoes_2015)
alts_2015[:5]

['A: desertificação das áreas afetadas; B: poluição dos rios temporários; C: queimadas dos remanescentes vegetais; D: desmatamento das matas ciliares; E: contaminação das águas subterrâneas',
 'A: melhoria da qualidade da produção industrial; B: redução da oferta de emprego nas zonas rurais; C: permissão ao trabalhador para controlar seus próprios horários; D: diminuição das exigências de esforço no trabalho com máquinas; E: ampliação do período disponível para a jornada de trabalho',
 'A: desenvolvimento do pensamento autônomo; B: obtenção de qualificação profissional; C: resgate de valores tradicionais.; D: realização de desejos pessoais; E: aumento da renda familiar',
 'A: elevado preço das mercadorias no comércio; B: aumento da demanda por produtos naturais; C: crescimento da produção de alimentos; D: hábito de adquirir derivados industriais.; E: uso de agrotóxicos nas plantações',
 'A: Justificar a dominação estamental / revoltas camponesas; B: Subverter a hierarquia social / cent

**2016**

In [191]:
questoes_2016, alts_2016 = get_alternatives(questoes_2016)
alts_2016[:5]

['A: consagração de relacionamentos afetivos; B: administração da independência interior; C: fugacidade do conhecimento empírico.; D: liberdade de expressão religiosa.; E: busca de prazeres efêmeros',
 'A: representação do legislativo com a fórmula do bipartidarismo; B: detenção de lideranças populares por crimes de subversão; C: presença de políticos com trajetórias no regime autoritário; D: prorrogação das restrições advindas dos atos institucionais; E: estabilidade da economia com o congelamento anual de preços',
 'A: Êxodo rural; B: Movimento pendular; C: Migração de retorno; D: Deslocamento sazonal; E: Ocupação de áreas centrais',
 'A: Frente pioneira; B: Zona de transição; C: Região polarizada; D: Área de conurbação; E: Periferia metropolitana',
 'A: concepção idealizada do território, entendido como geograﬁcamente indiferenciado; B: percepção corrente de uma ancestralidade comum às populações ameríndias; C: compreensão etnocêntrica acerca das populações dos territórios conquista

**2017**

In [192]:
questoes_2017, alts_2017 = get_alternatives(questoes_2017)
alts_2017[:5]

['A:  isolamento geográﬁco do Saara ocidental; B:  exploração intensiva de recursos naturais; C:  posição relativa nas redes de circulação; D:  tráﬁco transatlântico de mão de obra servil; E:  competição econômica dos reinos da região',
 'A:  ataque feito pelos japoneses à base militar americana de Pearl Harbor. ; B:  desencadeamento da Guerra Fria e de novas rivalidades entre nações; C:  morte de milhões de soldados nos combates da Segunda Guerra Mundial.; D:  execução de judeus e eslavos presos em guetos e campos de concentração nazistas; E:  lançamento de bombas atômicas em Hiroshima e Nagasaki pelas forças norte-americanas',
 'A:  fundamentação cientíﬁca de viés positivista; B:  convenção social de orientação normativa; C:  transgressão comportamental religiosa; D:  racionalidade de caráter pragmático; E:  inclinação de natureza passional',
 'A: modernização da educação escolar; B: atualização da disciplina moral cristã; C: divulgação de costumes aristocráticos.; D: socialização do

# Montando o dataset final

In [193]:
df_final = pd.concat([merge_quests_alts(df_2015, questoes_2015, alts_2015),
                      merge_quests_alts(df_2016, questoes_2016, alts_2016),
                      merge_quests_alts(df_2017, questoes_2017, alts_2017)], axis=0)

df_final.to_csv('df_15-17.csv')